# Advanced: Custom Data Integration with Chainladder AI Agent

This notebook demonstrates how to integrate your own custom data with the Chainladder AI Agent. We'll explore loading custom triangle data, using the agent to analyze it, and integrating the agent into your own actuarial workflows.

In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import chainladder as cl

# Add the project root to the path so we can import our package
sys.path.append('../..')

# Import the supervisor agent creator
from chainladder_agent.agents.supervisor import create_chainladder_supervisor
from chainladder_agent.tools.data_tools import register_custom_triangle

# Get API key
api_key = os.environ.get("OPENAI_API_KEY")
if not api_key:
    print("Warning: No OpenAI API key found. Please set your API key.")
else:
    print("API key is set.")

# Create the supervisor agent
supervisor = create_chainladder_supervisor(api_key=api_key)

## Helper Function for Agent Interaction

In [ ]:
def ask_agent(query, triangle_name=""):
    """Helper function to interact with the Chainladder AI Agent.
    
    Parameters:
        query (str): The question or instruction for the agent
        triangle_name (str): Optional name of the triangle dataset to use
        
    Returns:
        str: The agent's response
    """
    # Prepare the input
    input_data = {
        "messages": [{"role": "user", "content": query}],
        "selected_triangle": triangle_name
    }
    
    # Run the query
    result = supervisor.invoke(input_data)
    
    # Find the AI response
    ai_messages = []
    
    for message in result.get("messages", []):
        if hasattr(message, 'type') and message.type == 'ai':
            # Add this message to our collection
            if message.content and len(message.content) > 20:
                ai_messages.append((len(message.content), message.content))
    
    # Sort by content length and get the longest (likely most informative) response
    if ai_messages:
        ai_messages.sort(reverse=True)
        return ai_messages[0][1]
    else:
        return "No response received."

print("Helper function defined successfully.")

## 1. Creating a Custom Triangle

First, let's create a custom triangle from our own data. We'll create synthetic data for this example, but in practice, you would load your actual data from a CSV file or database.

In [ ]:
# Create synthetic data for a claims development triangle
np.random.seed(42)  # For reproducibility

# Define the dimensions of the triangle
n_origins = 5  # Number of accident years
n_devs = 6     # Number of development years

# Create empty dataframe
columns = [f"dev{i}" for i in range(n_devs)]
index = [f"2019+{i}" for i in range(n_origins)]
df = pd.DataFrame(index=index, columns=columns)

# Fill with random data representing cumulative claims
base_claims = 10000
growth_factor = 1.15  # Annual growth in claims
dev_pattern = [0.5, 0.75, 0.85, 0.95, 0.98, 1.0]  # Development pattern

for i, origin in enumerate(index):
    # Base ultimate for this origin period with growth
    ultimate = base_claims * (growth_factor ** i) 
    
    # Introduce some random variation
    ultimate = ultimate * np.random.uniform(0.9, 1.1)
    
    # Fill in the development pattern with some noise
    for j, dev in enumerate(columns):
        if i + j < n_origins + n_devs - 1:  # Only fill the triangle, not the future values
            factor = dev_pattern[j] * np.random.uniform(0.98, 1.02)
            df.loc[origin, dev] = round(ultimate * factor)

# Display the resulting triangle
display(df)

# Convert to a chainladder Triangle object
custom_triangle = cl.Triangle(df, origin=df.index, development=df.columns, cumulative=True)

# Register the custom triangle with the agent
register_custom_triangle("custom_property", custom_triangle)

print("Custom triangle created and registered with the agent as 'custom_property'.")

## 2. Exploring the Custom Triangle

Let's use the agent to explore and understand our custom triangle:

In [ ]:
exploration = ask_agent("Analyze this custom property triangle. Provide a summary of its key characteristics including the number of origin periods, development periods, and any notable patterns in the data.", "custom_property")
print(exploration)

## 3. Basic Analysis of the Custom Triangle

Now let's have the agent perform some basic analysis on our custom triangle:

In [ ]:
basic_analysis = ask_agent("Perform a basic Chain Ladder analysis on the custom property triangle. Show the development factors, ultimate losses, and IBNR by origin year.", "custom_property")
print(basic_analysis)

## 4. Visualizing the Custom Triangle

Let's create some visualizations of our custom triangle:

In [ ]:
visualization = ask_agent("Create visualizations for the custom property triangle. Include a heatmap of the triangle data and a chart showing the development patterns by origin year.", "custom_property")
print(visualization)

## 5. Method Comparison on Custom Data

Let's compare different actuarial methods on our custom triangle:

In [ ]:
method_comparison = ask_agent("Compare three different reserving methods (Chain Ladder, Bornhuetter-Ferguson, and Benktander) on the custom property triangle. Provide a table comparing the results and explain which method might be most appropriate for this data.", "custom_property")
print(method_comparison)

## 6. Sensitivity Analysis

Let's perform a sensitivity analysis on our custom triangle:

In [ ]:
sensitivity_analysis = ask_agent("Perform a sensitivity analysis on the development factors for the custom property triangle. Discuss how changes in the selected development factors could impact the ultimate loss estimates.", "custom_property")
print(sensitivity_analysis)

## 7. Loading Data from a CSV File

In practice, you would typically load your triangle data from a CSV file or database. Let's create an example CSV file and demonstrate how to load it:

In [ ]:
# Save our synthetic triangle to a CSV file
df.to_csv('custom_property_triangle.csv')
print("Saved triangle data to custom_property_triangle.csv")

# Now let's demonstrate loading this CSV file
df_loaded = pd.read_csv('custom_property_triangle.csv', index_col=0)
display(df_loaded)

# Convert to a chainladder Triangle object
loaded_triangle = cl.Triangle(df_loaded, origin=df_loaded.index, development=df_loaded.columns, cumulative=True)

# Register the loaded triangle with the agent
register_custom_triangle("loaded_property", loaded_triangle)

print("Loaded triangle from CSV and registered with the agent as 'loaded_property'.")

In [ ]:
loaded_analysis = ask_agent("Verify that the loaded_property triangle is identical to the custom_property triangle by comparing their key characteristics and running a basic Chain Ladder analysis on both.", "loaded_property")
print(loaded_analysis)

## 8. Handling Multiple Lines of Business

Let's create a second custom triangle for a different line of business and show how to analyze multiple triangles:

In [ ]:
# Create a second synthetic triangle for a different line of business
np.random.seed(43)  # Different seed for different patterns

# Same dimensions as before
df2 = pd.DataFrame(index=index, columns=columns)

# Different parameters for this line of business
base_claims = 5000  # Lower base claims
growth_factor = 1.25  # Higher growth
dev_pattern = [0.3, 0.6, 0.8, 0.9, 0.95, 1.0]  # Slower development pattern

for i, origin in enumerate(index):
    # Base ultimate for this origin period with growth
    ultimate = base_claims * (growth_factor ** i) 
    
    # Introduce some random variation
    ultimate = ultimate * np.random.uniform(0.85, 1.15)
    
    # Fill in the development pattern with some noise
    for j, dev in enumerate(columns):
        if i + j < n_origins + n_devs - 1:  # Only fill the triangle, not the future values
            factor = dev_pattern[j] * np.random.uniform(0.97, 1.03)
            df2.loc[origin, dev] = round(ultimate * factor)

# Display the second triangle
display(df2)

# Convert to a chainladder Triangle object
custom_triangle2 = cl.Triangle(df2, origin=df2.index, development=df2.columns, cumulative=True)

# Register the second custom triangle with the agent
register_custom_triangle("custom_liability", custom_triangle2)

print("Second custom triangle created and registered with the agent as 'custom_liability'.")

In [ ]:
multi_lob = ask_agent("Compare the development patterns between the custom_property and custom_liability triangles. Identify the key differences and how they might impact the reserving approach for each line of business.")
print(multi_lob)

## 9. Comparing Custom Data with Industry Benchmarks

Let's compare our custom data with industry benchmarks (represented by the sample triangles):

In [ ]:
benchmark_comparison = ask_agent("Compare the development patterns of our custom_property triangle with the industry patterns represented by the 'ukmotor' sample triangle. What are the key differences and similarities? What might these differences imply about our book of business?", "custom_property")
print(benchmark_comparison)

## 10. Custom Reporting

Let's generate a custom report for our portfolio of triangles:

In [ ]:
custom_report = ask_agent("Generate a comprehensive portfolio report for our custom_property and custom_liability triangles. Include summary statistics, development patterns, Chain Ladder results for each, and a combined portfolio view of total reserves. Format this as a professional report suitable for presentation to senior management.")
print(custom_report)

## 11. Integration with Custom Workflow

Let's demonstrate how to integrate the agent into a custom actuarial workflow:

In [ ]:
# Define a custom workflow function that uses the agent
def run_quarterly_reserve_analysis(triangle_name, confidence_level=0.75):
    """Perform a standardized quarterly reserve analysis workflow.
    
    Parameters:
        triangle_name (str): Name of the registered triangle to analyze
        confidence_level (float): Confidence level for risk margin calculation
        
    Returns:
        dict: Analysis results
    """
    results = {}
    
    # Step 1: Basic data validation and summary
    print("Step 1: Data Validation and Summary")
    data_summary = ask_agent(f"Validate the {triangle_name} data and provide a summary of key characteristics.", triangle_name)
    results['data_summary'] = data_summary
    print("\n")
    
    # Step 2: Development factor analysis
    print("Step 2: Development Factor Analysis")
    dev_analysis = ask_agent(f"Analyze the development factors for {triangle_name}. Identify any outliers and recommend appropriate selections.", triangle_name)
    results['development_analysis'] = dev_analysis
    print("\n")
    
    # Step 3: Method comparison
    print("Step 3: Method Comparison")
    method_analysis = ask_agent(f"Compare Chain Ladder, Bornhuetter-Ferguson, and Benktander methods for {triangle_name}. Recommend the most appropriate method with justification.", triangle_name)
    results['method_comparison'] = method_analysis
    print("\n")
    
    # Step 4: Uncertainty analysis
    print("Step 4: Uncertainty Analysis")
    uncertainty_analysis = ask_agent(f"Perform a Mack Chain Ladder analysis on {triangle_name} to quantify uncertainty. Calculate a risk margin at {confidence_level*100}% confidence level.", triangle_name)
    results['uncertainty_analysis'] = uncertainty_analysis
    print("\n")
    
    # Step 5: Executive summary
    print("Step 5: Executive Summary")
    executive_summary = ask_agent(f"Generate an executive summary of the reserve analysis for {triangle_name}, including recommended reserve and risk margin. Format this for senior management.", triangle_name)
    results['executive_summary'] = executive_summary
    
    return results

# Run the workflow on our custom property triangle
property_results = run_quarterly_reserve_analysis("custom_property")

## 12. Batch Processing Multiple Triangles

Let's demonstrate how to batch process multiple triangles:

In [ ]:
def batch_analyze_triangles(triangle_names):
    """Perform batch analysis on multiple triangles.
    
    Parameters:
        triangle_names (list): List of triangle names to analyze
        
    Returns:
        dict: Analysis results by triangle name
    """
    results = {}
    
    for name in triangle_names:
        print(f"\nAnalyzing {name}...\n")
        # Basic Chain Ladder analysis
        analysis = ask_agent(f"Perform a Chain Ladder analysis on {name} and summarize the key results (development factors, ultimate losses, and IBNR).", name)
        results[name] = analysis
        
    # Portfolio summary
    triangle_list = ", ".join(triangle_names)
    portfolio_summary = ask_agent(f"Provide a portfolio summary for the following triangles: {triangle_list}. Include total reserves and any insights about the overall portfolio.")
    results['portfolio_summary'] = portfolio_summary
    
    return results

# Run batch analysis on our custom triangles plus a sample triangle
batch_results = batch_analyze_triangles(["custom_property", "custom_liability", "ukmotor"])

# Display the portfolio summary
print("\n\nPORTFOLIO SUMMARY:")
print(batch_results['portfolio_summary'])

## Summary

In this advanced notebook, we've explored how to integrate custom data with the Chainladder AI Agent:

1. Creating custom triangles from different data sources
2. Registering custom triangles with the agent
3. Performing analyses on custom data
4. Comparing multiple lines of business
5. Benchmarking against industry standards
6. Generating custom reports
7. Integrating the agent into custom actuarial workflows
8. Batch processing multiple triangles

These capabilities enable you to leverage the Chainladder AI Agent for your specific business needs and integrate it into your existing actuarial processes. The agent provides a powerful interface for exploring, analyzing, and reporting on your custom triangle data, which can significantly enhance your actuarial analysis capabilities.